In [1]:
import numpy as np
puzzle_input = [109,424,203,1,21102,1,11,0,1105,1,282,21102,1,18,0,1105,1,259,2102,1,1,221,203,1,21101,31,0,0,1105,1,282,21101,38,0,0,1105,1,259,20102,1,23,2,21201,1,0,3,21102,1,1,1,21101,57,0,0,1106,0,303,2101,0,1,222,20101,0,221,3,21001,221,0,2,21101,259,0,1,21101,80,0,0,1106,0,225,21102,1,97,2,21102,1,91,0,1105,1,303,1201,1,0,223,20102,1,222,4,21101,259,0,3,21101,225,0,2,21102,1,225,1,21102,1,118,0,1106,0,225,21001,222,0,3,21102,1,21,2,21102,133,1,0,1106,0,303,21202,1,-1,1,22001,223,1,1,21101,0,148,0,1105,1,259,1201,1,0,223,20101,0,221,4,21001,222,0,3,21101,14,0,2,1001,132,-2,224,1002,224,2,224,1001,224,3,224,1002,132,-1,132,1,224,132,224,21001,224,1,1,21101,195,0,0,106,0,109,20207,1,223,2,20102,1,23,1,21102,-1,1,3,21101,0,214,0,1106,0,303,22101,1,1,1,204,1,99,0,0,0,0,109,5,2101,0,-4,249,21201,-3,0,1,21202,-2,1,2,21201,-1,0,3,21101,0,250,0,1106,0,225,22101,0,1,-4,109,-5,2105,1,0,109,3,22107,0,-2,-1,21202,-1,2,-1,21201,-1,-1,-1,22202,-1,-2,-2,109,-3,2105,1,0,109,3,21207,-2,0,-1,1206,-1,294,104,0,99,21202,-2,1,-2,109,-3,2105,1,0,109,5,22207,-3,-4,-1,1206,-1,346,22201,-4,-3,-4,21202,-3,-1,-1,22201,-4,-1,2,21202,2,-1,-1,22201,-4,-1,1,22102,1,-2,3,21101,0,343,0,1105,1,303,1105,1,415,22207,-2,-3,-1,1206,-1,387,22201,-3,-2,-3,21202,-2,-1,-1,22201,-3,-1,3,21202,3,-1,-1,22201,-3,-1,2,22101,0,-4,1,21102,1,384,0,1106,0,303,1105,1,415,21202,-4,-1,-4,22201,-4,-3,-4,22202,-3,-2,-2,22202,-2,-4,-4,22202,-3,-2,-3,21202,-4,-1,-2,22201,-3,-2,1,21202,1,1,-4,109,-5,2106,0,0]

In [2]:
# OPCODE INT MACHINE

def decode(c): # returns mode_a,mode_b,mode_c,opcode
    code_5="{:05d}".format(c)
    return int(code_5[0]),int(code_5[1]),int(code_5[2]),10*int(code_5[3])+int(code_5[4])

def code_to_dictcode(code):
    dictcode = {i: code[i] for i in range(0, len(code))}
    return dictcode

def get_dictcode(i, dictcode): # handle negative?
    if not i in dictcode:
        return 0
    return dictcode[i]

def set_dictcode(i, value, dictcode): # handle negative?
    dictcode[i] = value

# if one_output=False --> returns OUTPUT - array of outputs
# else
# returns output,dictcode,i,input_signals,input_index,relative_base
def opcode_machine(dictcode, i, input_signals,input_index,relative_base,one_output=True,DBG=False):
    
    code_i = get_dictcode(i,dictcode)
    mode_a,mode_b,mode_c,opcode = decode(code_i)
    
    output = []
    
    while(True):

        if (DBG==True): print(code_i,mode_a,mode_b,mode_c,opcode)
        
        if mode_c==0: # position mode
            i1=get_dictcode(i+1,dictcode)
        elif mode_c==1: # immediate mode
            i1=i+1
        elif mode_c==2: # relative mode
            i1=get_dictcode(i+1,dictcode)+relative_base

            
        if mode_b==0: # position mode
            i2=get_dictcode(i+2,dictcode)
        elif mode_b==1: # immediate mode
            i2=i+2
        elif mode_b==2: # relative mode
            i2=get_dictcode(i+2,dictcode)+relative_base
            
        if mode_a==0: # position mode
            i3=get_dictcode(i+3,dictcode)
        elif mode_a==1: # immediate mode
            i3=i+3
        elif mode_a==2: # relative mode
            i3=get_dictcode(i+3,dictcode)+relative_base
            
        code_i1 = get_dictcode(i1,dictcode)
        code_i2 = get_dictcode(i2,dictcode)
        code_i3 = get_dictcode(i3,dictcode)

        if (opcode==99):
            if (DBG==True): print("HALT")
            if (one_output):
                #return (['HALT',output],dictcode,i,input_signals,input_index,relative_base)
                return ('HALT',dictcode,i,input_signals,input_index,relative_base)
            else:
                return output
        elif (opcode== 1): # addition
            set_dictcode(i3, code_i1+code_i2, dictcode)
            i = i +4
        elif (opcode == 2): # multiplication
            set_dictcode(i3, code_i1*code_i2, dictcode)
            i = i +4
        elif (opcode == 3): # input
            set_dictcode(i1, input_signals[input_index], dictcode)
            i = i+2
            if (input_index<len(input_signals)-1):
                input_index += 1
            else:
                input_index += 0
                #print("no more input signals")
                if (one_output):
                    return (output,dictcode,i,input_signals,input_index,relative_base)
        elif (opcode == 4): # output
            output = output + [code_i1]
            if (DBG==True): print(code_i1)
            i = i+2
            if (one_output):
                return (output,dictcode,i,input_signals,input_index,relative_base)
        elif (opcode == 5): #jump-if-true
            if code_i1!=0:
                i = code_i2
            else:
                i=i+3
        elif (opcode == 6): #jump-if-false
            if code_i1==0:
                i = code_i2
            else:
                i=i+3
        elif (opcode == 7): #less than
            if code_i1<code_i2:
                set_dictcode(i3,1,dictcode)
            else:
                set_dictcode(i3,0,dictcode)
            i = i+4
        elif (opcode == 8): #equals
            if code_i1==code_i2:
                set_dictcode(i3,1,dictcode)
            else:
                set_dictcode(i3,0,dictcode)
            i = i+4
        elif (opcode == 9): # relative base update
            relative_base += code_i1
            i = i+2
                
        if(DBG):
            print("i",i)
            print("relative_base",relative_base)
            print("dictcode",dictcode)
        code_i = get_dictcode(i,dictcode)
        mode_a,mode_b,mode_c,opcode = decode(code_i)

    # we never get there
    return "NEVER"


In [12]:

def to_ascii(ccc):
    ret = []
    for c in ccc:
        ret.append(ord(c))
    ret.append(10)
    ret.append(10)
    return ret

def from_ascii(aaa):
    ret=""
    for a in aaa:
        ret += chr(a)
    return ret

# if one_output=True --> returns OUTPUT - array of outputs
# else
# returns output,dictcode,i,input_signals,input_index,relative_base
#def opcode_machine(dictcode, i, input_signals,input_index,relative_base,one_output=True,DBG=False):
    
count = 0
for xx in range(50):
    for yy in range(50):
        dictcode = code_to_dictcode(puzzle_input)

        i = 0
        relative_base=0
        input_signals=[xx,yy]
        input_index=0

        output = []

        #output, dictcode, \
        #i, input_signals, \
        #input_index, relative_base = opcode_machine(dictcode,i,
        #                                 input_signals,input_index,relative_base,
        #                                 one_output=True,DBG=False)

        output = opcode_machine(dictcode,i,
                                         input_signals,input_index,relative_base,
                                         one_output=False,DBG=False)

        count += output[0]
        print(output)
    
print ("COUNT",count)



[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
